In [1]:
import numpy as np
import pandas as pd
import json
import os

# DE - Cutoff 16 - 1024 Shots

In [ ]:
folder_path = r"C:\Users\Johnk\OneDrive\Desktop\PhD 2024\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\EM Comparison\Files\\"
potential_list = ['QHO', 'AHO', 'DW']
cutoff = 16

dataframes = {}

for potential in potential_list:

    fpath = folder_path + potential
    folders = [f for f in os.listdir(fpath) if os.path.isdir(os.path.join(fpath, f))]

    converged_runs = []
    mean_iters = []
    min_e = []
    delta_min_e = []
    median_e = []
    delta_median_e = []
    exact_e = []
    time_taken = []

    tol_list = []

    for f in folders:

        data_path = fpath + "\\{}\\{}_{}.json".format(f,potential, cutoff)
        
        with open(data_path, 'r') as file:
            data = json.load(file)

        success = sum(data['success'])
        if success > 0:
            tolerance = data['Optimizer']['tolerance']
            tol_list.append(tolerance)
            exact_min_e = np.min(data['exact_eigenvalues'])
            exact_e.append(exact_min_e)
            
            successful_indices = [i for i, success in enumerate(data['success']) if success]
            successful_results = [data['results'][i] for i in successful_indices]
            successful_iters = [data['num_iters'][i] for i in successful_indices]

            converged_runs.append(sum(data['success']))
                   
            mean_iters.append(int(np.round(np.mean(successful_iters))))
            min_e.append(np.min(successful_results))
            delta_min_e.append(abs(exact_min_e - np.min(successful_results)))
            median_e.append(np.median(successful_results))
            delta_median_e.append(abs(exact_min_e - np.median(successful_results)))
           
            time_taken.append(data['total_run_time'])



    df = pd.DataFrame({
        'Tolerance': tol_list,
        'Converged Runs': [f"{c}/100" for c in converged_runs],
        'Mean Iter': mean_iters,
        'VQE min E': [f"{num:.5e}" for num in min_e],
        'Delta min E': [f"{num:.5e}" for num in delta_min_e],
        'VQE median E': [f"{num:.5e}" for num in median_e],
        'Delta median E': [f"{num:.5e}" for num in delta_median_e],
        'Exact': [f"{num:.5e}" for num in exact_e],
        'Time': time_taken
    })

    df['Tolerance'] = [f"{float(tol):.0e}" for tol in df['Tolerance']]
    df = df.sort_values('Tolerance').reset_index(drop=True)

    dataframes[potential] = df


In [44]:
dataframes['QHO']

,Tolerance,Converged Runs,Mean Iter,VQE min E,Delta min E,VQE median E,Delta median E,Exact,Time
0,1e-01,100/100,42,0.00000e+00,0.00000e+00,4.88281e-04,4.88281e-04,0.00000e+00,0:07:53.742615
1,1e-02,100/100,55,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0:11:04.836117
2,1e-03,100/100,70,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0:13:23.791920
3,1e-04,80/100,125,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0:34:24.957507
4,1e-05,100/100,108,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0:20:07.842766
5,1e-06,100/100,108,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0:20:08.616835


In [45]:
dataframes['AHO']

,Tolerance,Converged Runs,Mean Iter,VQE min E,Delta min E,VQE median E,Delta median E,Exact,Time
0,1e-01,100/100,73,-1.16698e-03,1.53056e-13,-3.47317e-04,8.19659e-04,-1.16698e-03,0:12:09.224688
1,1e-02,100/100,127,-1.16698e-03,1.53056e-13,-1.16698e-03,1.53056e-13,-1.16698e-03,0:19:30.836384
2,1e-03,50/100,249,-1.16698e-03,1.53056e-13,-1.16698e-03,1.53056e-13,-1.16698e-03,0:57:26.515768


In [46]:
dataframes['DW']

,Tolerance,Converged Runs,Mean Iter,VQE min E,Delta min E,VQE median E,Delta median E,Exact,Time
0,1e-01,100/100,58,9.06035e-01,1.44359e-02,9.20170e-01,2.85710e-02,8.91599e-01,0:10:28.525166
1,1e-02,100/100,113,8.97299e-01,5.69985e-03,9.03088e-01,1.14883e-02,8.91599e-01,0:20:22.217244
2,1e-03,100/100,169,8.91641e-01,4.15834e-05,8.94340e-01,2.74054e-03,8.91599e-01,0:30:51.157226
3,1e-04,100/100,213,8.91641e-01,4.15428e-05,8.95370e-01,3.77034e-03,8.91599e-01,0:35:57.694317
4,1e-05,100/100,205,8.91641e-01,4.15834e-05,8.97299e-01,5.69976e-03,8.91599e-01,0:34:03.467950
5,1e-06,100/100,205,8.95240e-01,3.64018e-03,8.97299e-01,5.69976e-03,8.91599e-01,0:35:01.498676
